# Uploads

In [46]:
!rm -rf DNABERT-2-117M
! git lfs clone https://huggingface.co/zhihan1996/DNABERT-2-117M
%cd DNABERT-2-117M
! git fetch origin +refs/pull/*:refs/remotes/origin/pr/*
%cd ..

Cloning into 'DNABERT-2-117M'...
remote: Enumerating objects: 68, done.
remote: Total 68 (delta 0), reused 0 (delta 0), pack-reused 68 (from 1)
Unpacking objects: 100% (68/68), done.
/home/yamlibina/DNABERT-2-117M(1/1), 468 MB | 82 MB/s                           
/home/yamlibina


In [1]:
!git lfs clone https://github.com/vladislareon/z_dna

Cloning into 'z_dna'...
fatal: unable to access 'https://github.com/vladislareon/z_dna/': Failed connect to github.com:443; Connection timed out
Error(s) during clone:
git clone failed: exit status 128


In [2]:
!git clone https://github.com/Nazar1997/Sparse_vector.git

Cloning into 'Sparse_vector'...
^C


In [ ]:
!wget https://raw.githubusercontent.com/Boyle-Lab/Blacklist/master/lists/hg38-blacklist.v2.bed.gz

--2025-04-16 12:55:40--  https://raw.githubusercontent.com/Boyle-Lab/Blacklist/master/lists/hg38-blacklist.v2.bed.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5867 (5.7K) [application/octet-stream]
Saving to: ‘hg38-blacklist.v2.bed.gz’

hg38-blacklist.v2.b 100%[===================>]   5.73K  --.-KB/s    in 0s      

2025-04-16 12:55:40 (48.6 MB/s) - ‘hg38-blacklist.v2.bed.gz’ saved [5867/5867]



In [ ]:
!gzip -d hg38-blacklist.v2.bed.gz

In [1]:
!pip install transformers evaluate

In [2]:
!pip uninstall -y triton

In [1]:
!pip install wandb

Defaulting to user installation because normal site-packages is not writeable


# Imports

In [1]:
import torch

In [2]:
from transformers import AutoTokenizer, DataCollatorWithPadding, TrainingArguments, Trainer, DataCollatorForTokenClassification
from transformers.models.bert.configuration_bert import BertConfig
import numpy as np
import importlib
import evaluate

In [3]:
from DNABERT2.bert_layers import BertForTokenClassification

In [4]:
from joblib import load, dump, Parallel, delayed
import random
import os
from tqdm.notebook import tqdm, trange
import json
from Sparse_vector.sparse_vector import SparseVector
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split

import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader

In [5]:
from transformers_interpret import TokenClassificationExplainer
from collections import defaultdict

In [6]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cuda:0'

In [7]:
model_path_local = "DNABERT2"
model_path_hug = "zhihan1996/DNABERT-2-117M"

# Data preparation

In [8]:
RANDOM_SEED = 42

In [9]:
def set_random_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [10]:
set_random_seed(RANDOM_SEED)

## Loading data

In [11]:
chrom_names = [f'chr{i}' for i in list(range(1, 23)) + ['X', 'Y','M']]

def chrom_reader(chrom):
    files = sorted([i for i in os.listdir(f'z_dna/hg38_dna/') if f"{chrom}_" in i])
    return ''.join([load(f"z_dna/hg38_dna/{file}") for file in files])

DNA = {chrom:chrom_reader(chrom) for chrom in tqdm(chrom_names)}
lens_of_chroms = {chrom: len(DNA[chrom]) for chrom in DNA}
ZDNA = load('z_dna/hg38_zdna/sparse/ZDNA_cousine.pkl')

  0%|          | 0/25 [00:00<?, ?it/s]

In [ ]:
# Making a pickled blacklist

loc_dd = {chrm:SparseVector(lens_of_chroms[chrm]) for chrm in chrom_names}
blacklist = pd.read_csv('/content/hg38-blacklist.v2.bed', sep='\t', names=['chrom', 'begin', 'end', 'problem'])
blacklist.groupby('chrom')
for chrom, sub_df in tqdm(blacklist.groupby('chrom')):
    if chrom not in chrom_names:
        continue
    vec = np.zeros(lens_of_chroms[chrom])
    for inter in sub_df.values:
        vec[inter[1]:inter[2]+1] = np.maximum(vec[inter[1]:inter[2]+1], 1)
    loc_dd[chrom] = SparseVector(vec)

loc_dd

FileNotFoundError: [Errno 2] No such file or directory: '/content/hg38-blacklist.v2.bed'

In [ ]:
dump(loc_dd, 'blacklist_hg38_v2.pkl', 3)

In [14]:
black_list = load('blacklist_hg38_v2.pkl')

In [16]:
# Combine fix length intervals

width = 128

ints_in = []
ints_out = []


for chrm in chrom_names:
    for st in trange(0, ZDNA[chrm].shape - width, width):
        interval = [st, min(st + width, ZDNA[chrm].shape)]
        N_count = sum([bp == "N" for bp in DNA[chrm][interval[0]:interval[1]]])
        bl_count = black_list[chrm][interval[0]:interval[1]].sum()
        if N_count > width / 2 or bl_count > 0:
            continue
        else:
            if ZDNA[chrm][interval[0]: interval[1]].any():
                ints_in.append([chrm, int(interval[0]), int(interval[1]), 1])
            else:
                ints_out.append([chrm, int(interval[0]), int(interval[1]), 0])


print(len(ints_in))
print(len(ints_out))

ints_in_full = ints_in
ints_out_full = ints_out

  0%|          | 0/1944972 [00:00<?, ?it/s]

  0%|          | 0/1892136 [00:00<?, ?it/s]

  0%|          | 0/1549184 [00:00<?, ?it/s]

  0%|          | 0/1486051 [00:00<?, ?it/s]

  0%|          | 0/1418267 [00:00<?, ?it/s]

...

  0%|          | 0/397019 [00:00<?, ?it/s]

  0%|          | 0/1219069 [00:00<?, ?it/s]

  0%|          | 0/447089 [00:00<?, ?it/s]

  0%|          | 0/129 [00:00<?, ?it/s]

41889
22077059


In [17]:
ints_in = ints_in_full
ints_out = [ints_out_full[i] for i in np.random.choice(range(len(ints_out_full)),
                                                    size=len(ints_in) * 3, replace=False)]

print(len(ints_in))
print(len(ints_out))
equalized = np.array(ints_in + ints_out)


41889
125667


In [18]:
dump(equalized, 'hg38_kouzine_intervals_128.pkl', 3)

['hg38_kouzine_intervals_128 .pkl']

In [12]:
equalized = load('hg38_kouzine_intervals_128.pkl')

In [13]:
np.random.seed(RANDOM_SEED)

np.random.shuffle(equalized)
print(equalized)

[['chr11' '13466880' '13467008' '0']
 ['chr17' '38685952' '38686080' '0']
 ['chr6' '77021696' '77021824' '0']
 ...
 ['chr17' '70044160' '70044288' '0']
 ['chr3' '73998336' '73998464' '0']
 ['chr3' '126722944' '126723072' '0']]


In [14]:
y_stratify = [f"{elem[3]}_{elem[0]}" for i, elem in enumerate(equalized)]
train_intervals, test_intervals = train_test_split(equalized, test_size=0.2, shuffle=True, random_state=RANDOM_SEED, stratify=y_stratify)

y_train_stratify = [f"{elem[3]}_{elem[0]}" for i, elem in enumerate(train_intervals)]
train_intervals, val_intervals = train_test_split(train_intervals, test_size=0.3, shuffle=True, 
                                                  random_state=RANDOM_SEED, stratify=y_train_stratify)

In [15]:
len(train_intervals), len(val_intervals), len(test_intervals)

(93830, 40214, 33512)

## Create tokenizer

In [52]:
path_to_old_tokenizer = 'DNABERT-2-117M/tokenizer.json'
path_to_new_tokenizer = 'DNABERT2/tokenizer.json'
# We take a pre-trained DNABERT-2 tokenizer and remove tokens with length > 6 to make them more interpretable.

TOKEN_MAXLEN = 5
new_vocab = {}
new_merges = []
with open(path_to_old_tokenizer) as token_file:
    d = json.load(token_file)
    old_vocab = d['model']['vocab']
    for key, val in old_vocab.items():
        if (len(key) <= TOKEN_MAXLEN):
            new_vocab[key] = val

    merges = d['model']['merges']
    for merge in merges:
        a, b = merge.split()
        if len(a) + len(b) <= TOKEN_MAXLEN:
            new_merges.append(merge)

d['model']['vocab'] = new_vocab
d['model']['merges'] = new_merges

with open(path_to_new_tokenizer, 'w') as f:
    json.dump(d, f, indent=2)


In [16]:
tokenizer = AutoTokenizer.from_pretrained('DNABERT2')

In [17]:
label2id = tokenizer.vocab
id2label = {id : label for label, id in label2id.items()}

## Create dataset


In [18]:
class DNADataset(Dataset):
    def __init__(self, chroms,
                 dna_source,
                 labels, intervals, tokenizer, 
                 label_thr=None, multiclass=False,
                 return_intervals=False):
        self.chroms = chroms
        self.dna_source = dna_source
        self.labels = labels
        self.intervals = intervals
        self.label_thr = label_thr # for labeling based on percentage
        self.multiclass = multiclass # for multiclass token labeling
        self.return_intervals = return_intervals

    def __len__(self):
        return len(self.intervals)

    def __getitem__(self, idx):
        interval = self.intervals[idx]
        chrom = interval[0]
        begin = int(interval[1])
        end = int(interval[2])
        sequence = self.dna_source[chrom][begin:end].upper()
        special_tokens_label = -100

        # tokenize sequence
        # returns ['input_ids', 'token_type_ids', 'attention_mask']
        X = tokenizer(sequence, truncation=True, padding=True)
        y = [special_tokens_label]

        token_start = begin
        for token_id in X['input_ids'][1:-1]:
            token_len = len(id2label[token_id])
            token_end = token_start + token_len
            zdna_ys = self.labels[chrom][token_start:token_end]
            if self.label_thr is not None:
                ratio_1s = sum(zdna_ys) / token_len
                if ratio_1s >= self.label_thr:
                    y.append(1)
                else:
                    y.append(0)
            token_start = token_end
            
        y.append(special_tokens_label)
        y = torch.tensor(y, dtype=torch.int64)
        X['labels'] = y 
        if self.return_intervals:
            X['intervals'] = interval
        return X



In [19]:
train_dataset = DNADataset(chrom_names, DNA, ZDNA, train_intervals, tokenizer, label_thr=0.7)
val_dataset = DNADataset(chrom_names, DNA, ZDNA, val_intervals, tokenizer, label_thr=0.7)
test_dataset = DNADataset(chrom_names, DNA, ZDNA, test_intervals, tokenizer, label_thr=0.7)

In [20]:
len(train_dataset), len(val_dataset), len(test_dataset)

(93830, 40214, 33512)

In [22]:
from collections import Counter
c = Counter()
for i in range(len(train_dataset)):
    labels = train_dataset[i]['labels']
    c.update([l for l in labels.tolist() if l != -100])
print(c[0] / c[1]) 

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


24.546757764362706


# Loading model

In [71]:
from DNABERT2.bert_layers import BertForTokenClassification

model = BertForTokenClassification.from_pretrained(
    model_path_local,
    num_labels=2
)

model.to(device)

/home/yamlibina/DNABERT2/bert_layers.py:128: UserWarning: Unable to import Triton; defaulting MosaicBERT attention implementation to pytorch (this will reduce throughput when using this model).
  warnings.warn(
Some weights of the model checkpoint at DNABERT2 were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(4096, 768, padding_idx=0)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertUnpadAttention(
            (self): BertUnpadSelfAttention(
              (dropout): Dropout(p=0.0, inplace=False)
              (Wqkv): Linear(in_features=768, out_features=2304, bias=True)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
          )
          (mlp): BertGatedLinearUnitMLP(
            (gated_layers): Linear(in_features=768, o

# Training

In [72]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, padding=True)

In [73]:
import wandb
# wandb.finish()
wandb.login()
run_name = '10epochs_tokens0.7_128_maxtoken5'

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [74]:
os.environ["WANDB_PROJECT"] = "zdna2025"  # name your W&B project
os.environ["WANDB_LOG_MODEL"] = "checkpoint"
os.environ["WANDB_NAME"] = run_name

In [75]:
BATCH_SIZE=32
torch.cuda.empty_cache()

In [76]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    logits = logits[0]

    predictions = np.argmax(logits, axis=2)

    active_pred = []
    active_labels = []
    for prediction, label in zip(predictions, labels):
        for p, l in zip(prediction, label):
            if l != -100:
                active_pred.append(p)
                active_labels.append(l)
                

    mcc = evaluate.load("matthews_correlation")
    f1 = evaluate.load("f1")
    mcc_score = mcc.compute(predictions=active_pred, references=active_labels)["matthews_correlation"]
    f1_score = f1.compute(predictions=active_pred, references=active_labels)["f1"]

    return {"MCC": mcc_score, "F1": f1_score}

training_args = TrainingArguments(
    output_dir=run_name,
    learning_rate=5e-5,
    lr_scheduler_type="constant_with_warmup",
    warmup_ratio=0.1,
    optim='adamw_torch',
    label_names=["labels"],
    weight_decay=0.01,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    logging_strategy="steps",
    load_best_model_at_end=True,
    metric_for_best_model='F1',
    report_to="wandb"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)
trainer.can_return_loss = True
trainer.train()

Epoch,Training Loss,Validation Loss,Mcc,F1
1,0.082700,0.080423,0.732806,0.719499
2,0.067400,0.065150,0.805593,0.803825
3,0.058600,0.069551,0.793525,0.790069
4,0.047700,0.095018,0.842318,0.846678
5,0.037700,0.069372,0.826899,0.828023
6,0.032000,0.085262,0.832739,0.834526
7,0.025200,0.089017,0.843165,0.845861
8,0.022200,0.108634,0.847048,0.850924
9,0.020100,0.104964,0.841677,0.845583
10,0.016100,0.131325,0.855320,0.860041


wandb: Adding directory to artifact (./10epochs_tokens0.6_128_maxtoken5/checkpoint-2933)... Done. 3.9s
wandb: Adding directory to artifact (./10epochs_tokens0.6_128_maxtoken5/checkpoint-5866)... Done. 4.2s
wandb: Adding directory to artifact (./10epochs_tokens0.6_128_maxtoken5/checkpoint-8799)... Done. 4.0s
wandb: Adding directory to artifact (./10epochs_tokens0.6_128_maxtoken5/checkpoint-11732)... Done. 3.9s
wandb: Adding directory to artifact (./10epochs_tokens0.6_128_maxtoken5/checkpoint-14665)... Done. 3.6s
wandb: Adding directory to artifact (./10epochs_tokens0.6_128_maxtoken5/checkpoint-17598)... Done. 4.0s
wandb: Adding directory to artifact (./10epochs_tokens0.6_128_maxtoken5/checkpoint-20531)... Done. 4.2s
wandb: Adding directory to artifact (./10epochs_tokens0.6_128_maxtoken5/checkpoint-23464)... Done. 3.4s
wandb: Adding directory to artifact (./10epochs_tokens0.6_128_maxtoken5/checkpoint-26397)... Done. 3.5s
wandb: Adding directory to artifact (./10epochs_tokens0.6_128_maxto

TrainOutput(global_step=29330, training_loss=0.0468121421487511, metrics={'train_runtime': 6191.3313, 'train_samples_per_second': 151.551, 'train_steps_per_second': 4.737, 'total_flos': 2.537614165774632e+16, 'train_loss': 0.0468121421487511, 'epoch': 10.0})

In [77]:
trainer.state.best_model_checkpoint

'10epochs_tokens0.6_128_maxtoken5/checkpoint-29330'

In [ ]:
model.save_pretrained(f"dnabert2_{run_name}", from_pt=True)

# Evaluate nucleotide-wise

In [41]:
model = BertForTokenClassification.from_pretrained('10epochs_tokens0.6_128/checkpoint-best', num_labels=2)
model.to(device)

/home/yamlibina/DNABERT2/bert_layers.py:128: UserWarning: Unable to import Triton; defaulting MosaicBERT attention implementation to pytorch (this will reduce throughput when using this model).
  warnings.warn(


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(4096, 768, padding_idx=0)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertUnpadAttention(
            (self): BertUnpadSelfAttention(
              (dropout): Dropout(p=0.0, inplace=False)
              (Wqkv): Linear(in_features=768, out_features=2304, bias=True)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
          )
          (mlp): BertGatedLinearUnitMLP(
            (gated_layers): Linear(in_features=768, o

In [35]:
def eval_nucleotide(model, dataset):
    model.eval()
    y_true = []
    y_pred = []
    for idx in trange(0, len(dataset)):
        element = dataset[idx]
        element_torch = {k: torch.tensor(v).unsqueeze(0).to(device) for k, v in element.items()}
        token_ids = element['input_ids']
        tokens = [id2label[id_] for id_ in token_ids]

        interval = dataset.intervals[idx]
        chrom = interval[0]
        begin = int(interval[1])
        end = int(interval[2])

        with torch.no_grad():
            logits = model(**element_torch).logits

        predictions = torch.argmax(logits, dim=2)
        pred_nucleotide = []
        for i, pred in enumerate(predictions[0]):
            if i != 0 and i != len(predictions[0]) - 1:
                token_length = len(tokens[i])
                pred_nucleotide += [float(pred)] * token_length

        y_zdna = ZDNA[chrom][begin:end]

        y_true.append(y_zdna)
        y_pred.append(pred_nucleotide)
        
    return np.array(y_true), np.array(y_pred)

In [89]:
# max_len=5, thr=0.7
y_true_val, y_pred_val = eval_nucleotide(model, val_dataset)

from sklearn.metrics import matthews_corrcoef, f1_score, recall_score, precision_score

print("Val MCC:", matthews_corrcoef(y_true_val.flatten(), y_pred_val.flatten()))
print("Val F1:", f1_score(y_true_val.flatten(), y_pred_val.flatten()))
print("Val recall:", recall_score(y_true_val.flatten(), y_pred_val.flatten()))
print("Val precision:", precision_score(y_true_val.flatten(), y_pred_val.flatten()))


  0%|          | 0/40214 [00:00<?, ?it/s]

<ipython-input-35-749e5aeae918>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  element_torch = {k: torch.tensor(v).unsqueeze(0).to(device) for k, v in element.items()}


Val MCC: 0.8885354841823377
Val F1: 0.8916233599059517
Val recall: 0.9480934153786521
Val precision: 0.841502066373375


### Evaluate best model on test set

In [80]:
y_true_test, y_pred_test = eval_nucleotide(model, test_dataset)
from sklearn.metrics import matthews_corrcoef, f1_score, recall_score, precision_score
print("Test MCC:", matthews_corrcoef(y_true_test.flatten(), y_pred_test.flatten()))
print("Test F1:", f1_score(y_true_test.flatten(), y_pred_test.flatten()))
print("Test recall:", recall_score(y_true_test.flatten(), y_pred_test.flatten()))
print("Test precision:", precision_score(y_true_test.flatten(), y_pred_test.flatten()))

  0%|          | 0/33512 [00:00<?, ?it/s]

<ipython-input-35-749e5aeae918>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  element_torch = {k: torch.tensor(v).unsqueeze(0).to(device) for k, v in element.items()}


Test MCC: 0.88782615934438
Test F1: 0.8908354640315113
Test recall: 0.9483066058555245
Test precision: 0.839932238728173


# Interpretation

In [21]:
torch.cuda.empty_cache()

In [34]:
model = BertForTokenClassification.from_pretrained('10epochs_tokens0.7_128/checkpoint-best', num_labels=2)
model.to(device)

/home/yamlibina/DNABERT2/bert_layers.py:128: UserWarning: Unable to import Triton; defaulting MosaicBERT attention implementation to pytorch (this will reduce throughput when using this model).
  warnings.warn(


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(4096, 768, padding_idx=0)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertUnpadAttention(
            (self): BertUnpadSelfAttention(
              (dropout): Dropout(p=0.0, inplace=False)
              (Wqkv): Linear(in_features=768, out_features=2304, bias=True)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
          )
          (mlp): BertGatedLinearUnitMLP(
            (gated_layers): Linear(in_features=768, o

## TP index

In [23]:
from collections import defaultdict

def interpret_tp(model, dataset):
    model.eval()
    TP_counts = defaultdict(int)
    total_meetings_counts = defaultdict(int)

    special = ['[CLS]', '[SEP]', '[PAD]']

    for idx in trange(0, len(dataset)):
        element = dataset[idx]
        element_torch = {k: torch.tensor(v).unsqueeze(0).to(device) for k, v in element.items()}
        token_ids = element['input_ids']
        labels = element['labels']
        tokens = [id2label[id_] for id_ in token_ids]
        
        interval = dataset.intervals[idx]
        chrom = interval[0]
        begin = int(interval[1])
        end = int(interval[2])
        
        with torch.no_grad():
            logits = model(**element_torch).logits
            
        predictions = torch.argmax(logits, dim=2)[0]
    
        for tok_ind, token in enumerate(tokens):
            label = labels[tok_ind]
            pred = predictions[tok_ind]
            if token in special:
                continue
            total_meetings_counts[token] += 1
            if pred == label and label == 1:
                TP_counts[token] += 1
                
    return TP_counts, total_meetings_counts


In [24]:
TP_counts_val, total_counts_val = interpret_tp(model, test_dataset)

  0%|          | 0/33512 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
<ipython-input-23-bcf4d0729c6d>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  element_torch = {k: torch.tensor(v).unsqueeze(0).to(device) for k, v in element.items()}


In [25]:
tokens_map_val = {}
for token in TP_counts_val:
    if total_counts_val[token] > 0:
        tokens_map_val[token] = TP_counts_val[token] / total_counts_val[token]

In [26]:
sorted_tokens = {k: v for k, v in sorted(tokens_map_val.items(), key=lambda item: abs(item[1]), reverse=True)}
df_tp_val = pd.DataFrame(sorted_tokens.items(), columns=['token', 'score'])
df_cnt_val = pd.DataFrame(total_counts_val.items(), columns=['token', 'cnt'])
df_tp_val = df_tp_val.merge(df_cnt_val, on='token')
print(df_tp_val.head())
df_tp_val.to_csv("tokens_tp_ratio_cnt_maxlen5.csv", index=False)

   token     score   cnt
0  CACGC  0.717647   255
1   GCGC  0.639676  2470
2    CGC  0.486482  6658
3  GCGTG  0.453228   759
4  GCGCA  0.449664   745


## Integrated gradients (transformers-interpret)

In [27]:
np.random.seed(RANDOM_SEED)
intp_ids = np.random.randint(0, len(test_dataset), size = int(len(test_dataset) / 30))
interpret_dataset = DNADataset(chrom_names, DNA, ZDNA, test_intervals[intp_ids], tokenizer, label_thr=0.7)

In [28]:
from IPython.display import clear_output
from time import time

mce = TokenClassificationExplainer(model, tokenizer)

total_meetings_counts = defaultdict(int)
TP_counts = defaultdict(int)
positive_scores_sum = defaultdict(int)

model.eval()
for i in tqdm(range(len(interpret_dataset))):
   
    with torch.no_grad():
        element = interpret_dataset[i]
        element_torch = {k: torch.tensor(v).unsqueeze(0).to(device) for k, v in element.items()}
        token_ids = element['input_ids']
        tokens = [id2label[id_] for id_ in token_ids]
        interval = interpret_dataset.intervals[i]
        chrom = interval[0]
        begin = int(interval[1])
        end = int(interval[2])
        
        with torch.no_grad():
            logits = model(**element_torch).logits
        predictions = torch.argmax(logits, dim=2)
        
        sequence = DNA[chrom][begin:end].upper()
        word_attributions = mce(sequence)
        for tok_ind, token in enumerate(tokens):
            label = element['labels'][tok_ind]
            if label != -100:
                pred = int(predictions[0][tok_ind])
                if label == pred and label == 1:
                    # true positive
                    tokens_watch = sorted(word_attributions[token]['attribution_scores'], key=lambda x: x[1])[::-1][:10]
                    for helpful_token, helpful_token_score in tokens_watch:
                        total_meetings_counts[helpful_token] += 1
                        if pred == label and pred == 1:
                            TP_counts[helpful_token] += 1
                            positive_scores_sum[helpful_token] += helpful_token_score

 

  0%|          | 0/1117 [00:00<?, ?it/s]

<ipython-input-28-c753a6fe9459>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  element_torch = {k: torch.tensor(v).unsqueeze(0).to(device) for k, v in element.items()}


In [31]:
tokens_map = {}
for token, count in TP_counts.items():
    summa = positive_scores_sum[token]
    tokens_map[token] = summa / count
    
sorted_tokens = {k: v for k, v in sorted(tokens_map.items(), key=lambda item: abs(item[1]), reverse=True)}

df_intr_gr = pd.DataFrame(sorted_tokens.items(), columns=['token', 'score'])
df_counts = pd.DataFrame(TP_counts.items(), columns=['token', 'count'])
                 
df_intgr_merged = df_intr_gr.merge(df_counts, on='token')
# df_intgr_merged = df_intgr_merged.sort_values('Score', ascending=False)
df_intgr_merged.head()

,token,score,count
0,CTGTT,0.649018,4
1,GCGTG,0.593879,51
2,CGTG,0.520633,289
3,TACTA,0.516894,1
4,GCGC,0.500663,319


In [32]:
df_intgr_merged.to_csv('integrated_gradinents_scores_true_positive.csv')